## [ 데이터 병합/연결/조인 ]
- 여러 개의 Series/DataFrame 객체의 데이터들을 연결 합치기 방법

(1) 모듈로딩

In [1]:
import pandas as pd

(2) 데이터 준비

In [2]:
datas = {'name':[" 홍 길동", "이 나영", "마 징가", "베 토벤"], 
         "age":[10, 21, 73, 89],
         "reg_date":['2000/01/23', '2020/11/8', '1999/5/4', '1987/12/23']}

datas2 = [[" 홍 길동", "이 나영", "마 징가", "베 토벤"], 
         [10, 21, 73, 89],
         ['2000/01/23', '2020/11/8', '1999/5/4', '1987/12/23']]

(3) 데이터 저장

In [3]:
# Dict ==> DataFrame
df1 = pd.DataFrame(datas)
df1

,name,age,reg_date
0,홍 길동,10,2000/01/23
1,이 나영,21,2020/11/8
2,마 징가,73,1999/5/4
3,베 토벤,89,1987/12/23


In [4]:
# List ==> DataFrame
df2 = pd.DataFrame(datas2)
df2

,0,1,2,3
0,홍 길동,이 나영,마 징가,베 토벤
1,10,21,73,89
2,2000/01/23,2020/11/8,1999/5/4,1987/12/23


(4) 데이터 정보 확인

In [5]:
# 데이터의 전체 기본 정보 => info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3 non-null      object
 1   1       3 non-null      object
 2   2       3 non-null      object
 3   3       3 non-null      object
dtypes: object(4)
memory usage: 224.0+ bytes


(5) 데이터 전처리

- (5-1) 행 <=> 열 치환

In [6]:
df2 = df2.T
df2

,0,1,2
0,홍 길동,10,2000/01/23
1,이 나영,21,2020/11/8
2,마 징가,73,1999/5/4
3,베 토벤,89,1987/12/23


- (5-2) 실제 데이터와 타입 확인 후 형변환

In [7]:
df2.head()

,0,1,2
0,홍 길동,10,2000/01/23
1,이 나영,21,2020/11/8
2,마 징가,73,1999/5/4
3,베 토벤,89,1987/12/23


In [8]:
# 1번 컬럼 : object => 정수
df2[1]=df2[1].astype('uint8')

In [9]:
df2.dtypes

0    object
1     uint8
2    object
dtype: object

In [10]:
# 2번 컬럼 : object => datetime64[ns]
df2[2]=df2[2].astype('datetime64[ns]')

In [11]:
df2.dtypes

0            object
1             uint8
2    datetime64[ns]
dtype: object

- (5-3) 0번 컬럼의 이름을 성과 이름으로 분리하기

In [12]:
# 0번 컬럼만 추출 => Series
nameSR = df2[0]
nameSR

0     홍 길동
1     이 나영
2     마 징가
3     베 토벤
Name: 0, dtype: object

In [13]:
# "이 나영" 데이터를 추출 => 성과 이름으로 분리
nameSR[1]

'이 나영'

In [14]:
namesDF = nameSR.str.split(expand=True)
namesDF

,0,1
0,홍,길동
1,이,나영
2,마,징가
3,베,토벤


- (5-4) 두개의 DatafFrame을 컬럼방향으로 연결

In [15]:
pd.concat([df2, namesDF], axis='columns', ignore_index=True)

,0,1,2,3,4
0,홍 길동,10,2000-01-23,홍,길동
1,이 나영,21,2020-11-08,이,나영
2,마 징가,73,1999-05-04,마,징가
3,베 토벤,89,1987-12-23,베,토벤


- (5-5) 두개의 DataFrame을 행(row) 방향 데이터 연결

In [16]:
# 여러 개의 DF, SR을 연결 시에 모든 컬럼명/행인덱스만 연결 시켜주는 방식 ==> outer 방식
pd.concat([df2, namesDF])

,0,1,2
0,홍 길동,10,2000-01-23
1,이 나영,21,2020-11-08
2,마 징가,73,1999-05-04
3,베 토벤,89,1987-12-23
0,홍,길동,NaT
1,이,나영,NaT
2,마,징가,NaT
3,베,토벤,NaT


In [17]:
# 여러 개의 DF, SR을 연결 시에 동일 컬럼명/행인덱스만 연결 시켜주는 방식 ==> inner 방식
pd.concat([df2, namesDF], join='inner')

,0,1
0,홍 길동,10
1,이 나영,21
2,마 징가,73
3,베 토벤,89
0,홍,길동
1,이,나영
2,마,징가
3,베,토벤


In [18]:
df2.append(namesDF)        # FutureWarning

C:\Users\kdp\AppData\Local\Temp\ipykernel_21148\3120460443.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2.append(namesDF)        # FutureWarning


,0,1,2
0,홍 길동,10,2000-01-23
1,이 나영,21,2020-11-08
2,마 징가,73,1999-05-04
3,베 토벤,89,1987-12-23
0,홍,길동,NaT
1,이,나영,NaT
2,마,징가,NaT
3,베,토벤,NaT


### [ 병합 - (2) merge]
- 두 개의 DF에서 특정 컬럼을 기준으로 데이터를 합치는 것
- 두 DF의 합치는 기준이 되는 컬럼을 지정 필요
- 지정된 컬럼명이 없으면 동일한 컬럼명을 기준으로 합쳐짐

- (2-1) 데이터 준비

In [19]:
f1 = '../DATA/stock price.xlsx'
f2 = '../DATA/stock valuation.xlsx'

- (2-2) 데이터 저장 EXCEL ==> DataFrame

In [20]:
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2)

- (2-3) 데이터 확인

In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          13 non-null     int64  
 1   stock_name  13 non-null     object 
 2   value       11 non-null     float64
 3   price       11 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 544.0+ bytes


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      10 non-null     int64  
 1   name    10 non-null     object 
 2   eps     10 non-null     float64
 3   bps     10 non-null     int64  
 4   per     10 non-null     float64
 5   pbr     10 non-null     float64
dtypes: float64(3), int64(2), object(1)
memory usage: 608.0+ bytes


- (2-4) 데이터프레임 병합

- (2-4-1) merge()

In [23]:
# [기본] 두 개의 DF에서 동일 컬럼명을 찾아서 두 DF에 존재하는 컬럼명의 행 데이터만 합치기
pd.merge(df1, df2)

,id,stock_name,value,price,name,eps,bps,per,pbr
0,138040,메리츠금융지주,827.500000,NaN,메리츠금융지주,2122.333333,14894,6.313806,0.899691
1,130960,CJ E&M,58540.666667,98900.0,CJ E&M,6301.333333,54068,15.695091,1.829178
2,139480,이마트,239230.833333,254500.0,이마트,18268.166667,295780,13.931338,0.860437
3,145990,삼양사,82750.000000,82000.0,삼양사,5741.000000,108090,14.283226,0.758627
4,181710,NHN엔터테인먼트,NaN,NaN,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
5,185750,종근당,40293.666667,100500.0,종근당,3990.333333,40684,25.185866,2.470259
6,204210,모두투어리츠,3093.333333,3475.0,모두투어리츠,85.166667,5335,40.802348,0.651359
7,207940,삼성바이오로직스,NaN,91900.0,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


In [24]:
# [설정 => how 'outer' ] 두 개의 DF에서 동일 컬럼명을 찾아서 모두 합치기
pd.merge(df1, df2, how='outer')

,id,stock_name,value,price,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000.0,NaN,NaN,NaN,NaN,NaN
1,138040,메리츠금융지주,827.500000,NaN,메리츠금융지주,2122.333333,14894.0,6.313806,0.899691
2,130960,CJ E&M,58540.666667,98900.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
3,138250,엔에스쇼핑,14558.666667,13200.0,NaN,NaN,NaN,NaN,NaN
4,139480,이마트,239230.833333,254500.0,이마트,18268.166667,295780.0,13.931338,0.860437
5,142280,녹십자엠에스,468.833333,10200.0,NaN,NaN,NaN,NaN,NaN
6,145990,삼양사,82750.000000,82000.0,삼양사,5741.000000,108090.0,14.283226,0.758627
7,181710,NHN엔터테인먼트,NaN,NaN,NHN엔터테인먼트,2110.166667,78434.0,30.755864,0.827447
8,185750,종근당,40293.666667,100500.0,종근당,3990.333333,40684.0,25.185866,2.470259
9,192400,쿠쿠홀딩스,179204.666667,177500.0,NaN,NaN,NaN,NaN,NaN


In [25]:
# [설정 => how 'left' ] 두 개의 DF에서 동일 컬럼명을 찾아서 왼쪽 DF를 기준으로 합치기
pd.merge(df1, df2, how='left')

,id,stock_name,value,price,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000.0,NaN,NaN,NaN,NaN,NaN
1,138040,메리츠금융지주,827.500000,NaN,메리츠금융지주,2122.333333,14894.0,6.313806,0.899691
2,130960,CJ E&M,58540.666667,98900.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
3,138250,엔에스쇼핑,14558.666667,13200.0,NaN,NaN,NaN,NaN,NaN
4,139480,이마트,239230.833333,254500.0,이마트,18268.166667,295780.0,13.931338,0.860437
5,142280,녹십자엠에스,468.833333,10200.0,NaN,NaN,NaN,NaN,NaN
6,145990,삼양사,82750.000000,82000.0,삼양사,5741.000000,108090.0,14.283226,0.758627
7,181710,NHN엔터테인먼트,NaN,NaN,NHN엔터테인먼트,2110.166667,78434.0,30.755864,0.827447
8,185750,종근당,40293.666667,100500.0,종근당,3990.333333,40684.0,25.185866,2.470259
9,192400,쿠쿠홀딩스,179204.666667,177500.0,NaN,NaN,NaN,NaN,NaN


In [26]:
# [설정 => how 'right' ] 두 개의 DF에서 동일 컬럼명을 찾아서 오른쪽 DF를 기준으로 합치기
pd.merge(df1, df2, how='right')

,id,stock_name,value,price,name,eps,bps,per,pbr
0,130960,CJ E&M,58540.666667,98900.0,CJ E&M,6301.333333,54068,15.695091,1.829178
1,136480,NaN,NaN,NaN,하림,274.166667,3551,11.489362,0.887074
2,138040,메리츠금융지주,827.500000,NaN,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,139480,이마트,239230.833333,254500.0,이마트,18268.166667,295780,13.931338,0.860437
4,145990,삼양사,82750.000000,82000.0,삼양사,5741.000000,108090,14.283226,0.758627
5,161390,NaN,NaN,NaN,한국타이어,5648.500000,51341,7.453306,0.820007
6,181710,NHN엔터테인먼트,NaN,NaN,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
7,185750,종근당,40293.666667,100500.0,종근당,3990.333333,40684,25.185866,2.470259
8,204210,모두투어리츠,3093.333333,3475.0,모두투어리츠,85.166667,5335,40.802348,0.651359
9,207940,삼성바이오로직스,NaN,91900.0,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


In [27]:
# [설정 => how 'cross' ] 두 개의 DF에서 
pd.merge(df1, df2, how='cross')

,id_x,stock_name,value,price,id_y,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000.0,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,128940,한미약품,59385.666667,421000.0,136480,하림,274.166667,3551,11.489362,0.887074
2,128940,한미약품,59385.666667,421000.0,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,128940,한미약품,59385.666667,421000.0,139480,이마트,18268.166667,295780,13.931338,0.860437
4,128940,한미약품,59385.666667,421000.0,145990,삼양사,5741.000000,108090,14.283226,0.758627
...,...,...,...,...,...,...,...,...,...,...
125,207940,삼성바이오로직스,NaN,91900.0,161390,한국타이어,5648.500000,51341,7.453306,0.820007
126,207940,삼성바이오로직스,NaN,91900.0,181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
127,207940,삼성바이오로직스,NaN,91900.0,185750,종근당,3990.333333,40684,25.185866,2.470259
128,207940,삼성바이오로직스,NaN,91900.0,204210,모두투어리츠,85.166667,5335,40.802348,0.651359


In [28]:
idsr = df1.id
valuesr = df1.value

In [29]:
pd.concat([df1, idsr])

,id,stock_name,value,price,0
0,128940.0,한미약품,59385.666667,421000.0,NaN
1,138040.0,메리츠금융지주,827.500000,NaN,NaN
2,130960.0,CJ E&M,58540.666667,98900.0,NaN
3,138250.0,엔에스쇼핑,14558.666667,13200.0,NaN
4,139480.0,이마트,239230.833333,254500.0,NaN
5,142280.0,녹십자엠에스,468.833333,10200.0,NaN
6,145990.0,삼양사,82750.000000,82000.0,NaN
7,181710.0,NHN엔터테인먼트,NaN,NaN,NaN
8,185750.0,종근당,40293.666667,100500.0,NaN
9,192400.0,쿠쿠홀딩스,179204.666667,177500.0,NaN


In [30]:
pd.concat([valuesr, idsr], join='inner', axis=1)

,value,id
0,59385.666667,128940
1,827.500000,138040
2,58540.666667,130960
3,14558.666667,138250
4,239230.833333,139480
5,468.833333,142280
6,82750.000000,145990
7,NaN,181710
8,40293.666667,185750
9,179204.666667,192400


In [31]:
pd.concat([df1.iloc[0], df1.iloc[1]], join='inner')

id                  128940
stock_name            한미약품
value         59385.666667
price             421000.0
id                  138040
stock_name         메리츠금융지주
value                827.5
price                  NaN
dtype: object

In [32]:
df1.iloc[0]

id                  128940
stock_name            한미약품
value         59385.666667
price             421000.0
Name: 0, dtype: object

In [33]:
pd.DataFrame(df1.iloc[0]).T

,id,stock_name,value,price
0,128940,한미약품,59385.666667,421000.0


In [34]:
pd.concat([df1.iloc[0], df1.iloc[1]], join='inner', axis=1).T

,id,stock_name,value,price
0,128940,한미약품,59385.666667,421000.0
1,138040,메리츠금융지주,827.5,NaN


In [35]:
pd.concat([df1.iloc[0], df1])

,0,id,stock_name,value,price
id,128940,NaN,NaN,NaN,NaN
stock_name,한미약품,NaN,NaN,NaN,NaN
value,59385.666667,NaN,NaN,NaN,NaN
price,421000.0,NaN,NaN,NaN,NaN
0,NaN,128940.0,한미약품,59385.666667,421000.0
1,NaN,138040.0,메리츠금융지주,827.500000,NaN
2,NaN,130960.0,CJ E&M,58540.666667,98900.0
3,NaN,138250.0,엔에스쇼핑,14558.666667,13200.0
4,NaN,139480.0,이마트,239230.833333,254500.0
5,NaN,142280.0,녹십자엠에스,468.833333,10200.0


In [36]:
df3 = pd.DataFrame([[100]])
df3

,0
0,100


In [39]:
pd.concat([pd.DataFrame(df1.iloc[0]).T, df3], axis=0)

,id,stock_name,value,price,0
0,128940,한미약품,59385.666667,421000.0,NaN
0,NaN,NaN,NaN,NaN,100.0


In [40]:
pd.concat([pd.DataFrame(df1.iloc[0]).T, df3], axis=1)

,id,stock_name,value,price,0
0,128940,한미약품,59385.666667,421000.0,100
